In [100]:
import numpy as np
from scipy import signal

In [101]:
data=np.load('bci_3.npz')

In [102]:
data_2=data["X"]

In [84]:
# Apply notch filter
samp_freq = 1000  # Sample frequency (Hz)
notch_freq = 50.0  # Frequency to be removed from signal (Hz)
quality_factor = 20.0  # Quality factor
  
# Design a notch filter using signal.iirnotch
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, samp_freq)
  
# Compute magnitude response of the designed filter
freq, h = signal.freqz(b_notch, a_notch, fs=samp_freq)
  
#fig = plt.figure(figsize=(8, 6))


for i in range(data_2.shape[0]):
    for j in range(data_2.shape[1]):
        data_2[i, j, :] = signal.filtfilt(b_notch, a_notch, data_2[i,j,:])



In [103]:
#Selecting 8 electrodes as done in the text provided in 2nd best algorithm
data_2_new=data_2[:,(29,30,31,32,37,38,39,40),:]
data_2_new.shape

(278, 8, 3000)

In [128]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=8, max_iter=200)

#data_ica=[]
#for i in range(278):
data_ica=ica.fit_transform(data_2_new[0,:,:].T)


C:\Users\Ritwik\anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [104]:
#subsampling by 4 
data_2_sub=data_2_new
data_2_subs=np.zeros((278, 8, 750))
for i in range(278):
    for j in range(8):
        data_2_subs[i, j, :]=signal.resample(data_2_sub[i, j, :], 750)

data_2_subs.shape

(278, 8, 750)

In [105]:
#bandpass filter
b, a = signal.butter(2, 0.4, 'low', analog=False)
data_2_subs = signal.filtfilt(b, a, data_2_subs, axis=2)

In [106]:
#Extracting all the features and concatenating them 
final = np.array([])
for j in range(278):
    data_trial=data_2_subs[j,:,:].T
    #data_trial.shape

    data_trial_s1=data_trial[0:250,:]
    #print(data_trial_s1.shape)
    data_trial_s2=data_trial[250:500,:]
    #print(data_trial_s2.shape)
    data_trial_s3=data_trial[500:750,:]
    #print(data_trial_s3.shape)

    #AR Coefficients
    import statsmodels.api as sm
    #from statsmodels.datasets.sunspots import load
    #data = load()
    ARFV=np.array([])

    for i in range(8):
        rho1, sigma1 = sm.regression.linear_model.burg(data_trial_s1[:,i], order=2)
        rho2, sigma2 = sm.regression.linear_model.burg(data_trial_s2[:,i], order=2)
        rho3, sigma3 = sm.regression.linear_model.burg(data_trial_s3[:,i], order=2)
        ARFV=np.append(ARFV, (rho1, rho2, rho3))

    #print(ARFV) 

    #Haar wavelet
    import pywt
    HWDFV=np.array([])
    for i in range(8):
        (cA, cD) = pywt.dwt(data_trial[:,i], 'haar')
        HWDFV=np.append(HWDFV, cA)

    #Spectral Power estimates
    SPFV=np.array([])
    for i in range(8):
        f1, Pxx_den1 = signal.welch(data_trial_s1[:,i], 250)
        f2, Pxx_den2 = signal.welch(data_trial_s2[:,i], 250)
        f3, Pxx_den3 = signal.welch(data_trial_s3[:,i], 250)
        SPFV=np.append(SPFV, (Pxx_den1, Pxx_den2, Pxx_den3))

    #Concatenaton of All the feature vectors
    concated=np.concatenate((ARFV, HWDFV, SPFV), axis=None)
    concated=np.reshape(concated, (-1, 1))
    if j==0:
        final=concated
    else:
        final= np.hstack((final, concated))
    print(j)
print(final.shape)

C:\Users\Ritwik\anaconda3\lib\site-packages\scipy\signal\spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  warnings.warn('nperseg = {0:d} is greater than input length '


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [107]:
final=final.T
final.shape

(278, 6072)

In [108]:
#Labels of the training data
labels=data['events']
print(labels.shape)
labels=np.reshape(labels, (-1,))
print(labels.shape)

(278, 1)
(278,)


In [109]:
#Create an svm Classifier
from sklearn import svm

clf1 = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf1.fit(final, labels)

SVC(kernel='linear')

In [110]:
#Doing the same things for the test set

data_2_new_t=data['X_test'][:,(29,30,31,32,37,38,39,40),:]
data_2_new_t.shape

(100, 8, 3000)

In [111]:
#subsampling by 4 
data_2_sub_t=data_2_new_t
data_2_subs_t=np.zeros((100, 8, 750))
for i in range(100):
    for j in range(8):
        data_2_subs_t[i, j, :]=signal.resample(data_2_sub_t[i, j, :], 750)

data_2_subs_t.shape

(100, 8, 750)

In [112]:
#bandpass filter
b, a = signal.butter(2, 0.4, 'low', analog=False)
data_2_subs_t = signal.filtfilt(b, a, data_2_subs_t, axis=2)

In [113]:
final_t = np.array([])
for j in range(100):
    data_trial=data_2_subs_t[j,:,:].T
    #data_trial.shape

    data_trial_s1=data_trial[0:250,:]
    #print(data_trial_s1.shape)
    data_trial_s2=data_trial[250:500,:]
    #print(data_trial_s2.shape)
    data_trial_s3=data_trial[500:750,:]
    #print(data_trial_s3.shape)

    #AR Coefficients
    import statsmodels.api as sm
    #from statsmodels.datasets.sunspots import load
    #data = load()
    ARFV=np.array([])

    for i in range(8):
        rho1, sigma1 = sm.regression.linear_model.burg(data_trial_s1[:,i], order=2)
        rho2, sigma2 = sm.regression.linear_model.burg(data_trial_s2[:,i], order=2)
        rho3, sigma3 = sm.regression.linear_model.burg(data_trial_s3[:,i], order=2)
        ARFV=np.append(ARFV, (rho1, rho2, rho3))

    #print(ARFV) 

    #Haar wavelet
    import pywt
    HWDFV=np.array([])
    for i in range(8):
        (cA, cD) = pywt.dwt(data_trial[:,i], 'haar')
        HWDFV=np.append(HWDFV, cA)

    #Spectral Power estimates
    SPFV=np.array([])
    for i in range(8):
        f1, Pxx_den1 = signal.welch(data_trial_s1[:,i], 250)
        f2, Pxx_den2 = signal.welch(data_trial_s2[:,i], 250)
        f3, Pxx_den3 = signal.welch(data_trial_s3[:,i], 250)
        SPFV=np.append(SPFV, (Pxx_den1, Pxx_den2, Pxx_den3))

    #Concatenaton of All the feature vectors
    concated=np.concatenate((ARFV, HWDFV, SPFV), axis=None)
    concated=np.reshape(concated, (-1, 1))
    if j==0:
        final_t=concated
    else:
        final_t= np.hstack((final_t, concated))
    print(j)
print(final_t.shape)

C:\Users\Ritwik\anaconda3\lib\site-packages\scipy\signal\spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  warnings.warn('nperseg = {0:d} is greater than input length '


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
(6072, 100)


In [114]:
final_t=final_t.T
final_t.shape

(100, 6072)

In [115]:
#Loading the true labels for the test set from BCI 3 website
truelabels=np.loadtxt("true_labels.txt", delimiter="/n")

In [116]:
#Predicting the Test labels using previously trained model
y_pred_t = clf1.predict(final_t)

In [117]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(truelabels, y_pred_t))

Accuracy: 0.65
